<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import os
import pandas as pd
# import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import find_dotenv, load_dotenv 

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
# Используем пакет dotenv для чтения конфиденциальных данных из переменных окружения
# Данные содержатся в файле ".env". Он перечислен в .gitignore.
dotenv_path = find_dotenv() 
load_dotenv(dotenv_path)
DBNAME = os.getenv('DBNAME')
USER = os.getenv('USER')
HOST = os.getenv('HOST')
PASSWORD = os.getenv('PASSWORD')
PORT = os.getenv('PORT')

In [3]:
# Устанавливаем соединение с БД по рекомендации pandas
# При таком соединении предупреждения с username не выводятся.
engine = create_engine(
    "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}".format(
        dialect="postgresql",
        driver="psycopg2",
        username=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        database=DBNAME
    )
)
connection = engine.connect()

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
SELECT count(*)
FROM vacancies
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
SELECT count(*)
FROM employers
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
SELECT count(*)
FROM areas
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
SELECT count(*)
FROM industries
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

### Выводы по предварительному анализу данных:  
Рассматриваемая база данных содержит 49197 вакансий от 23501 работодателя из 1362 регионов и охватывает 294 сферы деятельности. База содержит вакансии из разных стран.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
SELECT count(vac.id) cnt,
       ars.name area
FROM vacancies vac
JOIN areas ars ON vac.area_id=ars.id
GROUP BY area
ORDER BY 1 DESC
limit 15
'''

In [13]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
5,1698,Екатеринбург
6,1670,Нижний Новгород
7,1415,Казань
8,1301,Краснодар
9,1144,Самара


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
SELECT count(*)
FROM vacancies
WHERE salary_from > 0
  OR salary_to > 0
'''

In [15]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
SELECT round(avg(salary_from)) salary_from,
       round(avg(salary_to)) salary_to
FROM vacancies
'''

In [17]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,salary_from,salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
SELECT count(id),
       schedule,
       employment
FROM vacancies
GROUP BY schedule,
         employment
ORDER BY 1 DESC
'''

In [19]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
SELECT count(id),
       experience
FROM vacancies
GROUP BY experience
ORDER BY 1
'''

In [21]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

### Выводы по детальному анализу вакансий:  
- Наибольшее количество вакансий сосредоточено в столицах и городах милионниках.  
- 49% вакансий содержат сведения о предполагаемой зарплате.  
- Средний показатель заработной платы от 71065 рублей до 110537 рублей. Что в целом отражает содержание набора высокооплачиваемых вакансий. 
- Самый востребованный распорядок работы и занятость - полный день/полная занятость - 72 %. На втором месте удаленная работа/полная занятость - 16%. 
- Наиболее востребованны вакансии с начальным опытом работы (от 1 года до 3 лет) - 53%, далее средний опыт работы (от 3 до 6 лет) - 29%. Без опыта работы - 15% вакансий. Профессионалы востребованы в 3 % вакансий.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
SELECT count(vac.id),
       emp.name
FROM vacancies vac
JOIN employers emp ON vac.employer_id=emp.id
GROUP BY emp.name
ORDER BY 1 DESC
'''

In [23]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
...,...,...
14761,1,им. Тьюринга
14762,1,LLC Novalab Tech
14763,1,Компания Мир упаковки
14764,1,Лифинцев Олег Иванович


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
SELECT ars.name,
       count(emp.id) employer,
       count(vac.id) vacan
FROM areas ars
LEFT JOIN employers emp ON emp.area=ars.id
LEFT JOIN vacancies vac ON vac.area_id=ars.id
WHERE vac.id IS NULL
GROUP BY ars.name
ORDER BY 2 DESC
'''


In [25]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,name,employer,vacan
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''
SELECT emp.name,
       count(DISTINCT vac.area_id) count_area_vac
FROM employers emp
JOIN vacancies vac ON vac.employer_id=emp.id
GROUP BY emp.name
ORDER BY 2 DESC
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df


,name,count_area_vac
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
SELECT count(emp.id) cnt
FROM employers emp
LEFT JOIN employers_industries ei ON ei.employer_id=emp.id
WHERE ei.industry_id IS NULL
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
SELECT emp.name,
       count(ei.industry_id) cnt
FROM employers emp
LEFT JOIN employers_industries ei ON ei.employer_id=emp.id
GROUP BY emp.id
HAVING count(ei.industry_id) = 4
ORDER BY 1
LIMIT 3
'''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,name,cnt
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
SELECT count(DISTINCT emp.id)
FROM employers emp
LEFT JOIN employers_industries ei ON ei.employer_id=emp.id
JOIN industries ind ON ind.id=ei.industry_id
WHERE ind.name='Разработка программного обеспечения'
'''

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'lxml') # Создаём объект BeautifulSoup
table = page.find("table",{"class":"standard sortable"}) # Находим таблицу с городами
links = table.find_all('a')[4:] # Находим ссылки в таблице, отсекаем не города
sity_name = tuple([link.get('title') for link in links]) # Извлекаем из заголовков название городов в кортеж
print(sity_name)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [35]:
# текст запроса
query_5_7 = f'''
WITH tmp AS
  (SELECT ars.name,
          count(vac.id) cnt
   FROM employers emp
   JOIN vacancies vac ON vac.employer_id=emp.id
   JOIN areas ars ON ars.id=vac.area_id
   WHERE (ars.name in {sity_name})
     AND (emp.name='Яндекс')
   GROUP BY ars.id
   ORDER BY 2)
SELECT tmp.name,
       tmp.cnt
FROM tmp
UNION ALL
SELECT 'Total',
       SUM(cnt)
FROM tmp
'''

In [36]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

### Выводы по анализу работодателей  
- Высокотехнологичные компании являются лидерами по количеству предлагаемых вакансий.  
- Наличие в таблице AREAS названий стран, областей не соответствуе описанию полей таблицы. Поле name должно содержать название города. Возможно требуется реструктуризация таблицы. 410(1,7%)работодателей действуют по всей России.  
- Лидерами по вакансиям в разных регионах являются высокотехнологичные компании(Из ТОП 5 - четыре ИТ компании).  
- 15% работодателей являются разработчиками программного обеспечения.  
- У компании Яндекс вакансии есть во всех городах милионниках.  

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''
SELECT count(name)
FROM vacancies
WHERE name ilike '%data%'
  OR name ilike '%%данн%%'
'''

In [38]:
# результат запроса
df = pd.read_sql_query(sql=text(query_6_1), con=connection)
# df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''
SELECT count(*)
FROM vacancies
WHERE (name ilike '%data scientist%'
       OR name ilike '%data science%'
       OR name ilike '%исследователь данных%'
       OR (name like '%ML%'
           AND name not ilike '%html%')
       OR name ilike '%machine learning%'
       OR name ilike '%машинн%обучен%')
  AND (name ilike '%junior%'
       OR experience = 'Нет опыта'
       OR employment = 'Стажировка')
'''

In [40]:
# результат запроса
# df = pd.read_sql_query(query_6_2, connection)
df = pd.read_sql_query(sql=text(query_6_2), con=connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''
SELECT count(*)
FROM vacancies
WHERE (name ilike '%data scientist%'
       OR name ilike '%data science%'
       OR name ilike '%исследователь данных%'
       OR (name ilike '%ml%'
           AND name not ilike '%html%')
       OR name ilike '%machine learning%'
       OR name ilike '%машинн%обучен%')
  AND (key_skills ilike '%sql%'
       OR key_skills ilike '%postgres%')
'''

In [42]:
# результат запроса
# df = pd.read_sql_query(query_6_3, connection)
df = pd.read_sql_query(sql=text(query_6_3), con=connection)
df

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''
SELECT count(*)
FROM vacancies
WHERE (name ilike '%data scientist%'
       OR name ilike '%data science%'
       OR name ilike '%исследователь данных%'
       OR (name ilike '%ml%'
           AND name not ilike '%html%')
       OR name ilike '%machine learning%'
       OR name ilike '%машинн%обучен%')
  AND (key_skills ilike '%python%')
'''

In [44]:
# результат запроса
# df = pd.read_sql_query(query_6_4, connection)
df = pd.read_sql_query(sql=text(query_6_4), con=connection)
df

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''
SELECT round(AVG(length(key_skills)-length(replace(key_skills, chr(9), '')) +1), 2)
FROM vacancies
WHERE (name ilike '%data scientist%'
       OR name ilike '%data science%'
       OR name ilike '%исследователь данных%'
       OR (name like '%ML%'
           AND name not ilike '%html%')
       OR name ilike '%machine learning%'
       OR name ilike '%машинн%обучен%')
'''

In [46]:
# результат запроса
# df = pd.read_sql_query(query_6_5, connection)
df = pd.read_sql_query(sql=text(query_6_5), con=connection)
df

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''
SELECT experience,
       round(avg(CASE
                     WHEN (salary_from > 0
                           AND salary_to > 0) THEN (salary_from+salary_to)/2
                     ELSE coalesce(salary_from, 0) + coalesce(salary_to, 0)
                 END), 0)
FROM vacancies
WHERE (name ilike '%data scientist%'
       OR name ilike '%data science%'
       OR name ilike '%исследователь данных%'
       OR (name like '%ML%'
           AND name not ilike '%html%')
       OR name ilike '%machine learning%'
       OR name ilike '%машинн%обучен%')
  AND (salary_from IS NOT NULL
       OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY 2
'''

In [48]:
# результат запроса
# df = pd.read_sql_query(query_6_6, connection)
df = pd.read_sql_query(sql=text(query_6_6), con=connection)
df
# закрываем соединение после окончания работы
connection.close()

***

# Выводы по предметному анализу
- Вакансий связанных с обработкой данных в базе 3,6%.  
- Вакансий для начинающих дата-сайентистов 11% (51 из 480).  
- Требования к знаниям баз данных и SQL у 48% (229 из 480) вакансий у дата-сайентистов.  
- Большинство вакансий дата-сайентистов 73% (351 из 480) требуют знания Python.  
- В среднем в вакансиях дата-сайентистов требуется 6,41 профессиональных навыков.  
- В среднем у вакансий дата-сайентистов наблюдается рост заработной палаты на 50% и выше при повышении опыта. ("Нет опыта" - 74643 руб., 53 % "От 1 года до 3 лет" - 139675 руб., 57% "От 3 до 6 лет" - 243115 руб.)

# Общий вывод по проекту

# подведем итог исследования, обобщите выводы
База содержит объем данных, позволяющий провести анализ и сделать выводы:
- Средняя заработная плата от 71065 до 110537 в целом выше показателей средней ЗП по России на 2024 год - 75034 руб.
- После прошедшей пандемии, работа на "полный день" опять востребована - 72 %
- Высокотехнологичные компании наиболее полно используют информационные ресурсы в своей деятельности, на пример "hh.ru".
- Дата сайентисты - это высоко оплачиваемые специалисты, с высоким требованием к профессиональным навыкам.  
Дополнительное исследование показало, что:
- Проект представлен в 44 странах.
- Комбинирую группы данных "ключевые навыки" и "сфера деятельности", можно получать срезы дополнительные аналитические группировки по другим признакам.

# Дополнительные исследования данных

Узнаем из скольки стран работодатели. Возмем список стран из [википедии](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2).


In [49]:
url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'lxml') # Создаём объект BeautifulSoup
table = page.find("table",{"class":"wikitable"}) # Находим таблицу со странами
links = table.find_all('a') # Находим ссылки в таблице
country_name = [links[idx].text for idx in range(1, len(links), 2)] # Извлекаем из заголовков название стран в список

In [50]:
query_ext_1 = f'''
WITH tmp AS (select unnest(array{country_name}) as country)
SELECT 
count(distinct ars.id) cnt
  FROM employers emp
  JOIN areas ars ON ars.id=emp.area
  JOIN tmp ON ars.name=tmp.country
'''


In [51]:
import psycopg2 # при использовании sqlalchemy - неявная ошибка
import warnings
warnings.filterwarnings('ignore', category=UserWarning) # Подавляем предупреждения
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)
# результат запроса
df = pd.read_sql_query(query_ext_1, connection)
df

,cnt
0,44


Выведем ТОП 3 наименования сфер деятельности, в навыках ваконсий которых требуется английский язык.

In [52]:
query_ext_2 = f'''
SELECT ind.name,
       count(ind.id)
FROM vacancies vcn
JOIN employers emp ON emp.id=vcn.employer_id
JOIN employers_industries empi ON emp.id=empi.employer_id
JOIN industries ind ON ind.id=empi.industry_id
WHERE key_skills ilike '%английский язык%'
GROUP BY ind.id
ORDER BY 2 DESC
LIMIT 10
'''

In [53]:
# результат запроса
df = pd.read_sql_query(query_ext_2, connection)
df

,name,count
0,Разработка программного обеспечения,1961
1,"Системная интеграция, автоматизации технологи...",1477
2,"Интернет-компания (поисковики, платежные систе...",559
3,Кадровые агентства,174
4,Банк,156
5,Консалтинговые услуги,130
6,Мобильная связь,117
7,"Маркетинговые, рекламные, BTL, дизайнерские, E...",100
8,"Электронно-вычислительная, оптическая, контрол...",68
9,"Финансово-кредитное посредничество (биржа, бро...",64


Три первых места связанны с информационными технологиями. 

In [54]:
# закрываем соединение после окончания работы
connection.close()